In [31]:
from pathlib import Path

import editdistance
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score

### This notebook computes the confidence intervals stratified per peptide edit distance to the training set using a nested bootstrap procedure. It should be run once for each model of the benchmark changing the *model* variable ('atm_tcr', 'rf', 'tulip' and 'xgb').

In [32]:
np.random.seed(42)
# These bins effectively turn off binning over TCR count
BINS_TCR = np.array([0,100000000]) # bins for TCR counts
# Subsample test to have no more than that number of test cases (different TCRs) per each test peptide
N_TCR_PER_PEP_TEST=np.Inf
## Count of bootstrap iterations inside each fold
N_BOOTS_INNER = 5000
## Count of bootstrap iterations when combining folds
N_BOOTS_OUTER = 5000
# Confidence interval level.
CI_ALPHA = 0.9
# Number of folds, default 1.
n_folds = 1 

In [33]:
# Name of the model ('atm_tcr', 'rf', 'tulip', 'xgb').
model = "rf" 
# Path to model results.
folder_results = Path(f"results_model/{model}/")
# Path to dataset reference csv.
ref_csv_path = Path("dataset/models_benchmark_dataset.csv")
# Path to save output.
output_path = Path(f"results_bootstrap/results_bootstrap_{model}.csv") 

In [34]:
def roc_auc(y_true: np.ndarray, y_score: np.ndarray) -> float:
    """Compute ROC-AUC using sklearn integration.
    
    Args:
        y_true (np.ndarray): True labels.
        y_score (np.ndarray): Predicted scores.
        method (str): Method to use, either skt (scikit-learn) or ustat. Default skt.
    
    Returns:
        auroc (float): ROC-AUC using U-statistics.     
    """
    if not len(np.unique(y_true)):
            raise AssertionError("Only one class present in y_true. ROC-AUC is not defined in that case.")
    return roc_auc_score(y_true, y_score)

In [35]:
def bin_auroc(df_test: pd.DataFrame, df_train: pd.DataFrame) -> pd.DataFrame:
    """Bin ROC-AUC per peptide edit distance and TCR count bin.
    
    Peptide edit distance is evaluated with the closest peptide to the training set.
    TCR count corresponds to the number of TCRs in the training set associated to the closest train peptide.
    
    Args:
        df_test (pd.DataFrame): Test dataframe containing peptide, labels and scores.
        df_train (pd.DataFrame): Train dataframe containing peptide, CDR3b sequences, labels and scores.
        
    Returns:
        df_auroc (pd.DataFrame): DataFrame with ROC-AUC per bin (peptide edit distance, TCR count)   
    """
    df_train_cnt = df_train.groupby("peptide_train").agg(tcr_count=("y_true","sum")).reset_index()

    # Perform cross-join and compute all edit distances test-train peptides.
    df_test_pep = df_test[["peptide"]].drop_duplicates()
    df_train_cnt["key"] = 1
    df_test_pep["key"] = 1
    df_pep_dist = pd.merge(df_train_cnt, df_test_pep, on='key').drop('key', axis=1)
    df_pep_dist["dist"] = df_pep_dist.apply(lambda row: editdistance.eval(row["peptide"],row["peptide_train"]),axis=1)

    # Only keep rows with the minimum values (maybe many instances).
    min_dist = df_pep_dist.groupby('peptide')['dist'].transform('min')
    df_pep_dist = df_pep_dist[df_pep_dist['dist'] == min_dist]

    # Count TCR examples per test peptide per distance in this fold and assign count to TCR count bin.
    df_pep_dist_sum = df_pep_dist.groupby(["peptide","dist"])["tcr_count"].sum().reset_index()
    df_pep_dist_sum["tcr_bin"] = np.digitize(df_pep_dist_sum["tcr_count"],BINS_TCR)

    # Join distance and tcr count bins with test examples dataframe for each peptide.
    df_pep_bins = pd.merge(df_pep_dist_sum[["peptide","dist","tcr_bin"]], 
                           df_test[["peptide","y_true","y_score"]], on='peptide')

    # Group by distance and TCR bins and compute ROCAUC over all examples in each bin.
    # We store count of all observations to later find weighted auroc average.
    # We store count of unique peptides with positive examples to later mask histogram.
    # Bins with low peptide counts and to show the count on the plot.
    df_auroc = df_pep_bins.groupby(['dist','tcr_bin']).\
    apply(lambda rows: pd.Series({'rocauc': roc_auc(rows['y_true'], rows['y_score']),
                                  'obs_cnt': len(rows['y_true']),
                                 'pep_cnt': len(rows[rows['y_true']>0]['peptide'].unique())})).\
    reset_index()
    return df_auroc

In [36]:
def bin_auroc_boot_iter(df_test: pd.DataFrame, df_train: pd.DataFrame, n_boots: int = 0) -> pd.DataFrame:
    """
    
    Args:
        df_test (pd.DataFrame): Test dataframe containing peptide, labels and scores.
        df_train (pd.DataFrame): Train dataframe containing peptide, CDR3b sequences, labels and scores.
        n_boots (int): Number of bootstrap iterations. Default 0.
    
    Returns:
        df_auroc (pd.DataFrame): DataFrame with ROC-AUC per bin (peptide edit distance, TCR count)       
    """
    df_train_cnt = df_train.groupby("peptide_train").agg(tcr_count=("y_true","sum")).reset_index()

    # Perform cross-join and compute all edit distances test-train peptides.
    df_test_pep = df_test[["peptide"]].drop_duplicates()
    df_train_cnt["key"] = 1
    df_test_pep["key"] = 1
    df_pep_dist = pd.merge(df_train_cnt, df_test_pep, on='key').drop('key', axis=1)
    df_pep_dist["dist"] = df_pep_dist.apply(lambda row: editdistance.eval(row["peptide"],row["peptide_train"]),axis=1)

    # Only keep rows with the minimum values (maybe many instances).
    min_dist = df_pep_dist.groupby('peptide')['dist'].transform('min')
    df_pep_dist = df_pep_dist[df_pep_dist['dist'] == min_dist]

    # Count TCR examples per test peptide per distance in this fold and assign count to TCR count bin.
    df_pep_dist_sum = df_pep_dist.groupby(["peptide","dist"])["tcr_count"].sum().reset_index()
    df_pep_dist_sum["tcr_bin"] = np.digitize(df_pep_dist_sum["tcr_count"],BINS_TCR)

    # Join distance and tcr count bins with test examples dataframe for each peptide.
    df_pep_bins = pd.merge(df_pep_dist_sum[["peptide","dist","tcr_bin"]], 
                           df_test[["peptide","y_true","y_score"]], on='peptide')

    assert len(df_pep_bins) == len(df_test), "Expected a single distance bin per each test peptide"
    
    # First loop is always the original sample (should not be used for variance estimation).
    # We can resample df_pep_bins instead of the original df_test because they correspond 1:1 (see assert above).
    # And because assignment of a test peptide to bins does not change if test observations are resampled
    # (assignment depends only on distance of peptide to train peptide and TCR counts of train peptides).
    for i_boot in range(n_boots+1):
        if i_boot == 0:
            df_pep_bins_boot = df_pep_bins
        else:
            df_pep_bins_boot = df_pep_bins.sample(n=len(df_pep_bins), replace=True)
        # Group by distance and TCR bins and compute ROCAUC over all examples in each bin.
        # We store count of all observations to later find weighted auroc average.
        # We store count of unique peptides with positive examples to later mask histogram.
        # bins with low peptide counts and to show the count on the plot
        df_auroc = df_pep_bins_boot.groupby(['dist','tcr_bin']).\
        apply(lambda rows: pd.Series({'rocauc': roc_auc(rows['y_true'], rows['y_score']),
                                      'obs_cnt': len(rows['y_true']),
                                     'pep_cnt': len(rows[rows['y_true']>0]['peptide'].unique())})).\
        reset_index()
        yield df_auroc

In [37]:
def auroc_over_folds(df_auroc_folds: pd.DataFrame) -> pd.DataFrame:
    """Aggregate ROC-AUC over the folds.
    
    Aggregate over all folds. ROCAUC is computed as an average weighted by counts of all observations in
    each fold (both positive and negative since they all contribute to ROC).
    
    Args:
        df_auroc_folds (pd.DataFrame): ROC-AUC and counts stratified per fold and bin.
    
    Returns:
        df_auroc (pd.DataFrame): ROC-AUC and counts aggregated over folds.    
    """
    def _aggr_rows(rows: pd.DataFrame) -> pd.Series:
        """Aggregation function per bin using a weighted average for the roc-auc and the sum for the counts.
        
        Args:
            rows (pd.DataFrame): Rows of the dataframe on which to aggregate.
        
        Returns:
            (pd.Series): Aggregated rows.        
        """
        rows = rows.dropna(subset=['rocauc'])
        return pd.Series({'rocauc': (rows['rocauc']*rows['obs_cnt']).sum()/rows['obs_cnt'].sum(),
                                  'obs_cnt': rows['obs_cnt'].sum(),
                                 'pep_cnt': rows['pep_cnt'].sum()})        
    df_auroc = df_auroc_folds.groupby(['dist','tcr_bin']).\
    apply(_aggr_rows).\
    reset_index()
    return df_auroc

In [38]:
def boot_fold_boots(n_folds: int, n_boots_inner: int, n_boots_outer: int) -> pd.DataFrame:
    """Generate dataframe with outer bootstrap indices.

    The idea is to randomly combine bootsrapped samples within each fold, and repeat generation
    of these combined tuples `n_boots_outer` times.
    Return value is a DataFrame with the following fields:
    `i_boot_inner` - for each fold, we select (with replacement) one bootstrap index that references 
    the internal fold-level bootstraps.
    `i_fold` iterates over folds
    `i_boot_outer` iterates of outer bootstrap loops
    The set with `i_boot_outer==0` is a special case: it contains `i_boot_innner=0` for every fold
    under a contract that `i_boot_inner=0` corresponds to non-bootsrapped original sample, and
    valid `i_boot_inner` runs from `0` to `n_boots_inner` inclusive. `i_boot_inner=0` will be only
    present in `i_boot_outer==0` set.
    Thus, `i_boot_outer` runs from `0` to `n_boots_outer` inclusive.
    
    Args:
        n_folds (int): Number of folds.
        n_boots_inner (int): Number of inner bootstraps.
        n_boots_outer (int): Number of outer bootstraps.
    
    Returns:
        boots (pd.DataFrame): DataFrame with outer bootstrap indices.
    """
    boots = []
    boots.append(pd.DataFrame({'i_fold':np.arange(1,n_folds+1,dtype=np.int_),
                               'i_boot_outer':np.int_(0),
                               'i_boot_inner':np.zeros(n_folds,dtype=np.int_)}))    
    for i_boot_outer in range(1,n_boots_outer+1):
        boots.append(pd.DataFrame({'i_fold':np.arange(1,n_folds+1,dtype=np.int_),
                                  'i_boot_outer':np.int_(i_boot_outer),
                                  'i_boot_inner':np.random.choice(range(1,n_boots_inner+1), size=n_folds, replace=True)}))
    boots = pd.concat(boots)
    return boots

In [39]:
def auroc_ci_from_boots(df_auroc_boot: pd.DataFrame, ci_alpha : float = 0.9) -> pd.DataFrame:
    """Aggregates ROC-AUC confidence intervals per bin.
    
    Args:
        df_auroc_boot (pd.DataFrame): DataFrame with ROC-AUC per bin for each bootstrap sample.
        ci_alpha (float): Confidence interval level. Default 0.9
    
    Returns:
        df_auroc (pd.DataFrame): DataFrame with ROC-AUC CI per bin computed over the bootstrap samples.
    """
    def _ci_metrics(rows: pd.DataFrame, ci_alpha: float = ci_alpha) -> pd.Series:
        ci_auc = np.quantile(rows['rocauc'], [0.5*(1-CI_ALPHA), 0.5*(1+CI_ALPHA)])
        return pd.Series({'rocauc_ci_low': ci_auc[0],
                          'rocauc_ci_high': ci_auc[1]})
    df_auroc = df_auroc_boot.groupby(['dist','tcr_bin']).\
    apply(_ci_metrics).\
    reset_index()
    return df_auroc


### Run bootstrap using provided paths

In [40]:
ref_df = pd.read_csv(ref_csv_path)

In [41]:
fold_boots = boot_fold_boots(n_folds, n_boots_inner = N_BOOTS_INNER, n_boots_outer = N_BOOTS_OUTER)

In [42]:
pep_test_cnt_check = 0
auroc_fold_check = []
obs_cnt_check = []
df_auroc_folds = []
for fold in range(1, n_folds+1):
    csv_path = folder_results / f"fold_{fold}.csv"
    df_test = pd.read_csv(csv_path)

    # Train contains all non-test peptides.
    df_train = ref_df[~ref_df['peptide'].isin(df_test['peptide'].unique())]
    assert set(df_train['peptide']).intersection(set(df_test['peptide'])) == set()

    # Make sure correct labelling of columns.
    # We need cdr3b, y_true, y_score, peptide.
    df_train = df_train.rename(columns={
        "CDR3b": "cdr3b",
        "binder": "y_true",
        "peptide": "peptide_train"
    })

    df_test = df_test.sample(n=len(df_test),replace=False).\
    groupby(['peptide','y_true']).head(N_TCR_PER_PEP_TEST)

    print(f'Ratio of positives in test fold {fold}: {df_test["y_true"].mean()}')

    # Accumulate data for sanity checks.
    pep_test_cnt_check += len(df_test['peptide'].unique())
    auroc_fold_check.append(roc_auc(y_true=df_test['y_true'],y_score=df_test['y_score']))
    obs_cnt_check.append(df_test.shape[0])

    for i_boot, df_auroc_fold in enumerate(bin_auroc_boot_iter(df_test=df_test,
                                                               df_train=df_train, 
                                                               n_boots=N_BOOTS_INNER)):
        df_auroc_fold['i_fold'] = fold
        df_auroc_fold['i_boot'] = i_boot
        df_auroc_folds.append(df_auroc_fold)

auroc_fold_check = np.asarray(auroc_fold_check)
obs_cnt_check = np.asarray(obs_cnt_check)

df_auroc_folds = pd.concat(df_auroc_folds)
# Shift tcr_bin to match expected offset of the plotting code.
df_auroc_folds['tcr_bin'] -= 1

# Drop nan ROCAUC records in case we had cells with only one class.
df_auroc_folds = df_auroc_folds[~df_auroc_folds['rocauc'].isna()]

Ratio of positives in test fold 1: 0.5


In [43]:
# Iterate over outer bootstrap samples.
df_auroc_boot = []
for i_boot_outer in range(N_BOOTS_OUTER+1):
    fold_boot_i = fold_boots[fold_boots['i_boot_outer']==i_boot_outer]
    fold_boot_i = fold_boot_i.rename(columns={'i_boot_inner':'i_boot'})
    df_auroc_folds_boot_i = pd.merge(df_auroc_folds,
                                     fold_boot_i[['i_fold','i_boot']],on=['i_fold','i_boot'])
    # Compute AUROC aggregated over folds within bins.
    df_auroc_boot_i = auroc_over_folds(df_auroc_folds_boot_i)
    df_auroc_boot_i['i_boot'] = i_boot_outer
    df_auroc_boot.append(df_auroc_boot_i)

df_auroc_boot = pd.concat(df_auroc_boot)

In [44]:
# Aggregate results.
df_auroc_ci = auroc_ci_from_boots(df_auroc_boot=df_auroc_boot[df_auroc_boot['i_boot']!=0],ci_alpha=CI_ALPHA)
df_auroc = pd.merge(df_auroc_boot[df_auroc_boot['i_boot']==0].drop('i_boot',axis=1),
                    df_auroc_ci,on=['dist','tcr_bin'])
df_auroc

,dist,tcr_bin,rocauc,obs_cnt,pep_cnt,rocauc_ci_low,rocauc_ci_high
0,1,0,0.439831,504.0,90.0,0.397426,0.481831
1,2,0,0.479855,188.0,32.0,0.410133,0.551811
2,3,0,0.423802,110.0,33.0,0.331442,0.513316
3,4,0,0.463178,888.0,131.0,0.429935,0.495317
4,5,0,0.487494,2706.0,436.0,0.468946,0.505694
5,6,0,0.487963,2554.0,444.0,0.469417,0.507394
6,7,0,0.532619,492.0,76.0,0.491474,0.574614
7,8,0,0.510802,144.0,11.0,0.431831,0.589231
8,9,0,0.371173,56.0,4.0,0.242417,0.501288


In [45]:
# Save results.
df_auroc.to_csv(output_path ,index=False)